In [1]:
!pip install --upgrade langchain langchain-community langchain-openai langchain-neo4j

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

In [3]:
from langchain_neo4j import Neo4jGraph

# Neo4j 연결 URL, 사용자명, 비밀번호를 지정합니다.
graph = Neo4jGraph(
    url="bolt://localhost:7687",  # Neo4j URL, Neo4j Bolt 프로토콜 사용
    username="neo4j",             # Neo4j 사용자명
    password=" "           # Neo4j 비밀번호 입력
)

In [4]:
graph.refresh_schema() # 새로운 노드(Node) 타입 또는 관계(Relationship)가 추가되었을 때, 반영하기 위해 실행
print(graph.schema)

Node properties:
Team {Name: STRING}
Player {Name: STRING}
Wage {Euro: FLOAT}
Relationship properties:

The relationships:
(:Player)-[:Part_of]->(:Team)
(:Player)-[:Paid]->(:Wage)


In [7]:
from langchain_neo4j import GraphCypherQAChain # 자연어를 Cypher 쿼리로 변환하여 Neo4j에서 검색을 수행
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
chain = GraphCypherQAChain.from_llm(
    graph=graph, # Neo4j 데이터베이스 객체를 연결
    llm=llm, 
    verbose=True,  # 실행 과정을 로그로 출력
    return_direct=True, # LangChain이 해석한 Cypher 쿼리 결과를 바로 반환
    allow_dangerous_requests=True) # 고급 Cypher 쿼리 실행 허용 (복잡한 질의 가능)
response = chain.invoke({"query": "Manchester United에 소속된 모든 선수를 알려줘"}) # 자연어 질문
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Player)-[:Part_of]->(t:Team {Name: "Manchester United"})
RETURN p.Name


> Finished chain.


{'query': 'Manchester United에 소속된 모든 선수를 알려줘',
 'result': [{'p.Name': 'Karim Benzema'},
  {'p.Name': 'Lionel Messi'},
  {'p.Name': 'Robert Lewandowski'},
  {'p.Name': 'Kylian Mbappé'},
  {'p.Name': 'Cristiano Ronaldo'},
  {'p.Name': 'Neymar'},
  {'p.Name': 'Sergio Ramos'},
  {'p.Name': 'Mohamed Salah'},
  {'p.Name': 'Eden Hazard'},
  {'p.Name': 'Gerard Piqué'}]}

In [8]:
response = chain.invoke({"query": "Manchester United 선수 한 명과 급여를 알려줘"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Player)-[:Part_of]->(t:Team {Name: "Manchester United"}), (p)-[:Paid]->(w:Wage)
RETURN p.Name, w.Euro
LIMIT 1


> Finished chain.


{'query': 'Manchester United 선수 한 명과 급여를 알려줘',
 'result': [{'p.Name': 'Karim Benzema', 'w.Euro': 70833.28422284627}]}

In [9]:
# 참고용도
response = chain.invoke({"query": "Manchester United에 소속된 선수 한 명과 그 선수의 급여를 알려줘"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Player)-[:Part_of]->(t:Team {Name: "Manchester United"}), (p)-[:Paid]->(w:Wage)
RETURN p.Name, w.Euro
LIMIT 1


> Finished chain.


{'query': 'Manchester United에 소속된 선수 한 명과 그 선수의 급여를 알려줘',
 'result': [{'p.Name': 'Karim Benzema', 'w.Euro': 70833.28422284627}]}